In [9]:
import json
import pandas as pd
import re
import string
from unicodedata import normalize

In [10]:
# plik sample to wycinek duzeo jsona .har ktory sobie troche w vimie obrobilem
with open('./sample.json', 'r') as f:
    text = f.read()

In [11]:
text[:100]

'            "[{\\"id\\":1642072642225,\\"date\\":1641942000000,\\"massif\\":\\"aiguilles d arves\\",\\"sommet'

In [14]:
data = json.loads(text)
# ech kiepsko jednak obrobilem :P
data = json.loads(data)
data

[{'id': 1642072642225,
  'date': 1641942000000,
  'massif': 'aiguilles d arves',
  'sommet': "Aiguille de l'Epaisseur",
  'itineraire': '',
  'description': "Après s'être décalé trop sur la gauche par rapport à l'itinéraire à la montée, j'ai vu une fissure partir au dessus de mes skis. En faisant demi-tour j'ai vu que la fissure courait de ma position jusqu'à une cinquantaine de mètre plus haut.\r\nUne bonne plaque à vent. ",
  'orientation': 'E',
  'caracteristique': '',
  'origine_principale': 'Ski touring, uphill',
  'origine_secondaire': '',
  'type_ecoulement_principal': '',
  'qualite_neige': '',
  'risque_meteo_france': 'Inconnu',
  'latitude': 45.13092028612608,
  'longitude': 6.357382659110359,
  'thumbnail': '/attached-files/thumbnails/1642072461149-IMG20220112120845.jpg',
  'imagesNb': 2,
  'isPrivate': False,
  'tags': '',
  'updated': 1642074353000,
  'altitude_depart': 3100,
  'altitude_arrivee': 0},
 {'id': 1642001062328,
  'date': 1641942000000,
  'massif': '',
  'somme

In [18]:
df = pd.DataFrame(data)
df.head()

,id,date,massif,sommet,itineraire,description,orientation,caracteristique,origine_principale,origine_secondaire,...,risque_meteo_france,latitude,longitude,thumbnail,imagesNb,isPrivate,tags,updated,altitude_depart,altitude_arrivee
0,1642072642225,1641942000000,aiguilles d arves,Aiguille de l'Epaisseur,,Après s'être décalé trop sur la gauche par rap...,E,,"Ski touring, uphill",,...,Inconnu,45.130920,6.357383,/attached-files/thumbnails/1642072461149-IMG20...,2,False,,1642074353000,3100,0
1,1642001062328,1641942000000,,,Mont Joly,Descente du Pcheu au Mont Joly. \r\nDéclenchem...,E,,,,...,Inconnu,45.825902,6.693000,/attached-files/thumbnails/1642000995871-6792F...,1,False,,1642013122000,2500,1180
2,1642005733893,1641855600000,Haute Maurienne,Pointe de Vallonbrun,,"Petite avalanche partie spontanément, probable...",SE,Wind slab,,,...,3 - MARQUE,45.317667,6.937318,/attached-files/thumbnails/1642005733893-16420...,2,False,,1642005978000,2630,2240
3,1641931532871,1641855600000,Belledonne,pointe du sciallet,"Sommet du sciallet par l'éperon est, permettan...",Declenchement de la plaque par 1 skieur. \r\nD...,E,,"Ski touring, downhill",,...,Inconnu,45.212236,6.013909,/attached-files/thumbnails/1641931326001-inbou...,3,False,,1641931532000,2300,2200
4,1641918131562,1641855600000,Mont Thabor,Roche de la Pelle,,Avalanche spontanée partie très probablement d...,NE,Soft slab,,,...,3 - MARQUE,45.136030,6.551387,/attached-files/thumbnails/1641918131562-16419...,5,False,,1641918700000,2820,2450


In [19]:
# tak pisalem to na odpierdziel :P
def flatten_strings_columns(df, cols):
    def clean_txt(t):
        t = str(t)
        t = (
            normalize('NFD', t)
            .encode('ascii', 'ignore')
            .decode('UTF-8')
        )
        t =  re.sub(r'\n', ' ', t)
        return t
    
    for col in cols:
        # df[col] = df[col].apply(clean_txt)
        # line = line.decode('UTF-8')
        df[col] = df[col].apply(lambda x: x.strip() if isinstance(x, str) else x)
        df[col] = df[col].apply(lambda x: normalize('NFD', x).encode('ascii', 'ignore') if isinstance(x, str) else x)
        df[col] = df[col].apply(lambda x: x.decode('UTF-8') if isinstance(x, str) else x)
        df[col] = df[col].apply(lambda x: re.sub(r'\n', ' ', x) if isinstance(x, str) else x)
    return df

In [20]:
df = pd.DataFrame(data)
df = flatten_strings_columns(df, ['description', 'caracteristique'])
df.to_csv('./data-avalanche.csv',index=False)

kopiuje csv do folderu data (tam bedzie nasze "repo danych")